In [181]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import PowerTransformer, StandardScaler, OrdinalEncoder

# First I will do all the dropping and renaming

In [182]:
X_train = pd.read_csv("train.csv")
y_train = X_train["Survived"]
X_train = X_train.drop(["PassengerId", "Name", "Ticket", "Cabin", "Survived"], axis=1)

# The magic function

In [183]:

# def separate(df):
#     return df[["Age","Fare"]], df[["Pclass", "Sex", "SibSp", "Parch", "Embarked"]]

# le = pd.read_pickle("le.pickle")
# pf = pd.read_pickle("pf.pickle")
# ps = pd.read_pickle("ps.pickle")
# lr = pd.read_pickle("lr.pickle")
# def transform(X):
#     X = pd.DataFrame([X])
#     X = X.rename(columns={0: 'Age', 1: 'Fare', 2:"Pclass",3:"Sex",4:"SibSp",5:"Parch",6:"Embarked"})

#     train_cont_toy,train_discr_toy = separate(X_train)
#     train_discr_toy_no_col = le.transform(train_discr_toy)
#     train_discr_toy = pd.DataFrame(train_discr_toy_no_col, columns = train_discr_toy.columns)
    
#     train_discr_toy = train_discr_toy.fillna(train_discr_toy.median())


#     train_cont_toy = pf.transform(train_discr_toy)


    

#     X_test_toy = pd.concat([pd.DataFrame(train_cont_toy), pd.DataFrame(train_discr_toy).astype("int16")], axis=1)
#     X_test_toy = ps.transform(X_test_toy)
#     y_pred = lr.predict(X_test_toy)



    

    
    

# Splitting into two arrays

In [184]:
def separate(df):
    return df[["Age","Fare"]], df[["Pclass", "Sex", "SibSp", "Parch", "Embarked"]]

In [185]:
train_cont,train_discr = separate(X_train)

# Filling the NANs

In [186]:
train_cont = train_cont.fillna(train_cont.mean())




le = OrdinalEncoder()
train_discr_no_col = le.fit_transform(train_discr)
train_discr = pd.DataFrame(train_discr_no_col, columns = train_discr.columns)

train_discr = train_discr.fillna(train_discr.median())
pd.to_pickle(le, "le.pickle")

# Poly the conts

In [187]:
pf = PolynomialFeatures(3)
train_cont = pf.fit_transform(train_cont)
train_discr.head()
pd.to_pickle(pf,"pf.pickle")

In [188]:
train_discr.isna().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

# Merge them back together

In [189]:


X_train = pd.concat([pd.DataFrame(train_cont), pd.DataFrame(train_discr).astype("int16")], axis=1)


# Now do all the scaling

In [190]:
ps = PowerTransformer()
X_train = ps.fit_transform(X_train)
pd.to_pickle(ps, "ps.pickle")

C:\Users\eitan.shirman\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\eitan.shirman\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


TypeError: cannot pickle 'module' object

# Finally, regression

In [ ]:
lr = LogisticRegression()

lr.fit(X_train, y_train)
pd.to_pickle(lr, "lr.pickle")

LogisticRegression()

In [ ]:
pd.DataFrame(X_train).tail()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
886,0.0,-0.164262,-0.294907,-0.162789,-0.236321,-0.281706,-0.162603,-0.218777,-0.257680,-0.279785,-0.542051,0.737695,-0.679956,-0.560253,0.610411
887,0.0,-0.798775,0.527906,-0.798775,0.121263,0.555182,-0.797641,-0.145333,0.316746,0.557258,-1.485485,-1.355574,-0.679956,-0.560253,0.610411
888,0.0,0.042001,0.288885,0.043555,0.275173,0.316823,0.043505,0.234304,0.296473,0.319257,0.856623,-1.355574,1.375734,1.846856,0.610411
889,0.0,-0.241577,0.527906,-0.240180,0.363851,0.555182,-0.239895,0.222136,0.455607,0.557258,-1.485485,0.737695,-0.679956,-0.560253,-1.780520
890,0.0,0.215254,-0.812349,0.216753,-0.480006,-0.837872,0.216535,-0.313998,-0.624600,-0.840242,0.856623,0.737695,-0.679956,-0.560253,-1.236339
